In [59]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import f1_score, classification_report, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from xgboost import XGBClassifier


In [60]:
df_train = pd.read_csv('combined_train_with_labels.csv.gz')

df_test = pd.read_csv('combined_test_with_labels.csv.gz')

In [61]:
df_train

,id,energy_label,psd_label_low_avse,psd_label_high_avse,psd_label_dcr,psd_label_lq,tp0,ED,HWP,LQ80,...,time_to_main_peak,time_to_peak,current_width,tdrift10,tdrift50,tdrift99,tfr,peak_count,gbn,bpr
0,0_train_0,582.364295,False,True,True,True,957,3409.0,2120.0,-635333.796311,...,85,85,0.126727,32.0,65.0,85.0,0.142357,3,1.198436,0.059642
1,1_train_0,250.159995,False,True,True,True,948,3404.0,2004.0,-289756.893085,...,87,87,0.136277,0.0,60.0,87.0,0.151160,3,1.381123,0.061975
2,2_train_0,1212.323954,False,True,False,True,965,3411.0,2125.0,-379843.029134,...,95,95,0.236836,35.0,67.0,95.0,0.142606,3,1.361857,0.050813
3,3_train_0,240.878110,False,True,True,False,927,3408.0,2098.0,-252673.626844,...,116,116,0.171464,0.0,90.0,116.0,0.133192,2,1.165654,0.062954
4,4_train_0,285.124189,False,True,True,False,958,3406.0,2037.0,-317761.453582,...,94,94,0.225977,0.0,61.0,94.0,0.145504,7,1.390918,0.055013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1039995,1039995_train_15,210.834626,True,True,True,True,949,3403.0,2048.0,-240759.341995,...,90,90,0.119532,0.0,55.0,90.0,0.151747,6,1.008092,0.062825
1039996,1039996_train_15,380.170340,True,True,True,True,949,3404.0,2045.0,-417622.302428,...,82,82,0.121399,0.0,57.0,79.0,0.150305,3,1.245492,0.062706
1039997,1039997_train_15,370.109563,False,True,True,True,947,3409.0,2063.0,-407730.168602,...,102,102,0.168929,0.0,77.0,102.0,0.142950,2,1.149456,0.062153
1039998,1039998_train_15,98.258524,True,True,True,False,921,3400.0,2001.0,-116570.568893,...,92,92,0.122313,0.0,71.0,92.0,0.153135,5,1.192639,0.065264


In [62]:
df_train.columns

Index(['id', 'energy_label', 'psd_label_low_avse', 'psd_label_high_avse',
       'psd_label_dcr', 'psd_label_lq', 'tp0', 'ED', 'HWP', 'LQ80', 'PPR',
       'SCA', 'ND80', 'current_skewness', 'tail_slope_no_pz',
       'spectral_centroid_power', 'current_kurtosis', 'tail_slope',
       'total_power', 'time_to_main_peak', 'time_to_peak', 'current_width',
       'tdrift10', 'tdrift50', 'tdrift99', 'tfr', 'peak_count', 'gbn', 'bpr'],
      dtype='object')

In [63]:
df_train['energy_label']

0           582.364295
1           250.159995
2          1212.323954
3           240.878110
4           285.124189
              ...     
1039995     210.834626
1039996     380.170340
1039997     370.109563
1039998      98.258524
1039999     281.715768
Name: energy_label, Length: 1040000, dtype: float64

In [64]:
type(df_train['psd_label_low_avse'][0])

numpy.bool

In [65]:
df_train_reg = df_train[
    (df_train["psd_label_low_avse"] == 1) &
    (df_train["psd_label_high_avse"] == 1) &
    (df_train["psd_label_dcr"] == 1) &
    (df_train["psd_label_lq"] == 1)
]


In [66]:
df_test_reg = df_test[
    (df_test["psd_label_low_avse"] == 1) &
    (df_test["psd_label_high_avse"] == 1) &
    (df_test["psd_label_dcr"] == 1) &
    (df_test["psd_label_lq"] == 1)
]


In [67]:
df_train_reg

,id,energy_label,psd_label_low_avse,psd_label_high_avse,psd_label_dcr,psd_label_lq,tp0,ED,HWP,LQ80,...,time_to_main_peak,time_to_peak,current_width,tdrift10,tdrift50,tdrift99,tfr,peak_count,gbn,bpr
5,5_train_0,202.118818,True,True,True,True,938,3401.0,2016.0,-2.238298e+05,...,86,86,0.121210,0.0,62.0,86.0,0.148161,2,1.333229,0.066141
6,6_train_0,855.765685,True,True,True,True,953,3413.0,2110.0,-9.533368e+05,...,112,112,0.125181,60.0,88.0,106.0,0.136972,1,1.463301,0.061205
8,8_train_0,1212.688940,True,True,True,True,961,3412.0,2089.0,-1.336303e+06,...,113,113,0.117900,65.0,82.0,105.0,0.140185,1,1.217066,0.065910
11,11_train_0,2334.785962,True,True,True,True,961,3412.0,2061.0,-2.494320e+06,...,119,119,0.122090,74.0,92.0,117.0,0.139700,1,1.488288,0.062823
14,14_train_0,2130.987393,True,True,True,True,980,3401.0,2057.0,-2.373392e+06,...,46,46,0.121277,14.0,21.0,43.0,0.157380,1,2.648069,0.072427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1039985,1039985_train_15,238.270546,True,True,True,True,966,3401.0,2024.0,-2.676853e+05,...,77,77,0.124577,0.0,26.0,77.0,0.164503,2,1.653406,0.070927
1039986,1039986_train_15,230.961021,True,True,True,True,929,3408.0,2032.0,-2.581743e+05,...,116,116,0.116108,0.0,94.0,116.0,0.138498,1,1.165428,0.069788
1039990,1039990_train_15,299.894902,True,True,True,True,932,3407.0,2021.0,-3.529948e+05,...,119,119,0.114861,36.0,84.0,119.0,0.138221,4,1.135231,0.066935
1039995,1039995_train_15,210.834626,True,True,True,True,949,3403.0,2048.0,-2.407593e+05,...,90,90,0.119532,0.0,55.0,90.0,0.151747,6,1.008092,0.062825


In [68]:
df_test_reg

,id,energy_label,psd_label_low_avse,psd_label_high_avse,psd_label_dcr,psd_label_lq,tp0,ED,HWP,LQ80,...,time_to_peak,spectral_centroid_power,current_width,tdrift10,tdrift50,tdrift99,tfr,peak_count,gbn,bpr
0,2395098_test_0,1167.174731,True,True,True,True,967,3407.0,2036.0,-1.300536e+06,...,77,108.796954,0.122269,34.0,55.0,76.0,0.148591,1,1.282821,0.062497
3,2395101_test_0,238.918902,True,True,True,True,930,3408.0,2053.0,-2.760460e+05,...,110,109.196803,0.116946,38.0,88.0,110.0,0.138597,2,1.226275,0.067184
5,2395103_test_0,238.613614,True,True,True,True,915,3410.0,2079.0,-2.555422e+05,...,154,110.325605,0.127910,0.0,127.0,154.0,0.134678,4,1.463728,0.060105
9,2395107_test_0,206.709993,True,True,True,True,958,3404.0,2056.0,-2.315807e+05,...,76,108.583390,0.119982,0.0,52.0,76.0,0.149437,3,3.362198,0.063002
10,2395108_test_0,8.909674,True,True,True,True,861,3422.0,1396.0,-1.453120e+04,...,374,541.252499,0.119396,0.0,0.0,374.0,0.517327,40,1.197393,0.167289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389992,2785090_test_5,2289.794166,True,True,True,True,971,3409.0,2050.0,-2.514208e+06,...,101,111.661949,0.123581,52.0,72.0,93.0,0.145278,1,1.406514,0.061910
389994,2785092_test_5,238.390197,True,True,True,True,966,3405.0,2057.0,-2.570958e+05,...,92,107.914168,0.122891,0.0,48.0,92.0,0.154106,4,1.107573,0.060723
389996,2785094_test_5,452.840234,True,True,True,True,952,3406.0,2058.0,-5.194853e+05,...,93,108.447129,0.115417,36.0,59.0,83.0,0.148058,3,2.268535,0.064586
389997,2785095_test_5,344.740556,True,True,True,True,948,3406.0,2036.0,-3.684673e+05,...,125,108.862927,0.124933,0.0,80.0,125.0,0.140774,3,1.217066,0.062375


In [69]:
print("Before cut:", len(df_train))
print("After cut:", len(df_train_reg))
print("Fraction kept:", len(df_train_reg) / len(df_train))


Before cut: 1040000
After cut: 395329
Fraction kept: 0.38012403846153847


In [70]:
label_cols = [
    "psd_label_low_avse",
    "psd_label_high_avse",
    "psd_label_dcr",
    "psd_label_lq"
]

non_feature_cols = ["id", "energy_label"] + label_cols


In [71]:
feature_cols = [c for c in df_train.columns if c not in non_feature_cols]

In [72]:
X_train = df_train_reg[feature_cols]
y_train = df_train_reg["energy_label"]

X_test = df_test_reg[feature_cols]
y_test = df_test_reg["energy_label"]

In [73]:
print("n_features:", len(feature_cols))
print("X_train shape:", X_train.shape)
print("Any NaNs in X_train:", X_train.isna().any().any())


n_features: 23
X_train shape: (395329, 23)
Any NaNs in X_train: False


In [74]:
# pipeline for linear regression
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("lr", LinearRegression())
])

pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('lr', LinearRegression())])

In [75]:
y_pred = pipe.predict(X_test)

mse = mean_squared_error(y_test, y_pred)

print("Test MSE:", mse)

Test MSE: 3534.342024899047


In [77]:
rmse = np.sqrt(mse)
print("Test RMSE:", rmse)


Test RMSE: 59.45033242042509
